In [42]:
import joblib
import numpy as np
import pandas as pd

# 1. ASSET LOADING 
try:
    hybrid_model = joblib.load('riskbridge_hybrid_model.pkl')
    scaler = joblib.load('riskbridge_scaler.pkl')
    model_columns = joblib.load('riskbridge_features.pkl')
    training_medians = joblib.load('riskbridge_medians.pkl')
    print(" System Check: Assets Loaded Successfully.")
except Exception as e:
    print(f" Error: {e}")

def riskbridge_final_auditor():
    print("      RISKBRIDGE: THE NUCLEAR AUDITOR")
    print("      Strict Credit Risk Enforcement")
    
    # 2. INPUT COLLECTION
    loan_amnt = float(input("1. Enter Loan Amount ($): "))
    annual_inc = float(input("2. Enter Annual Income ($): "))
    fico = float(input("3. Enter FICO Score: "))
    dti = float(input("4. Enter Debt-to-Income Ratio: "))

    # 3. INITIALIZE WITH MEDIANS
    input_df = pd.DataFrame(columns=model_columns)
    input_df.loc[0] = [training_medians.get(col, 0) for col in model_columns]

    # 4. HARD-ALIGNMENT & NUCLEAR OVERRIDES
    input_df['loan_amnt'] = loan_amnt
    input_df['annual_inc_log'] = np.log1p(annual_inc)
    input_df['fico_range_low'] = fico
    input_df['dti'] = dti
    input_df['last_fico_range_high'] = fico + 2 

    # --- THE "CONSERVATIVE" BASELINE ---
    input_df['inq_last_6mths'] = 3           
    input_df['acc_open_past_24mths'] = 5     
    input_df['total_rev_hi_lim'] = 10000     
    input_df['tot_cur_bal'] = 15000          
    input_df['term_encoded'] = 0.25          
    input_df['grade_encoded'] = 0.45 

    # --- DTI SENSITIVITY ENFORCEMENT (Placed BEFORE Prediction) ---
    if dti > 40:
        input_df['inq_last_6mths'] += 2          
        input_df['verification_status_encoded'] = 0.30  
        if loan_amnt > 15000:
             input_df['grade_encoded'] = 0.55   

    # 5. SCALE & PREDICT
    # We ensure the data is aligned and cast to float before scaling
    input_df = input_df[model_columns].astype(float)
    scaled_values = scaler.transform(input_df)
    scaled_df = pd.DataFrame(scaled_values, columns=model_columns)
    
    prob = hybrid_model.predict_proba(scaled_df)[:, 1][0]
    risk_score = prob * 100
    
    status = "APPROVED " if risk_score < 40 else "REJECTED "
    print(f" FINAL DECISION: {status}")
    print(f" CALCULATED RISK: {risk_score:.2f}%")

if __name__ == "__main__":
    riskbridge_final_auditor()

 System Check: Assets Loaded Successfully.
      RISKBRIDGE: THE NUCLEAR AUDITOR
      Strict Credit Risk Enforcement


1. Enter Loan Amount ($):  12000
2. Enter Annual Income ($):  60000
3. Enter FICO Score:  590
4. Enter Debt-to-Income Ratio:  28


 FINAL DECISION: REJECTED 
 CALCULATED RISK: 78.04%
